In [2]:
import json
import scipy.sparse as ss
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import scipy.io as sio
import random
np.random.seed(123)

In [22]:
foil_train = json.load(open('foil_new/FOIL-LINGv0.5_train2017.json'))



foil_train_VERB = [s for s in foil_train['annotations'] if s['POS'] == 'VERB'] + [s for s in foil_train['annotations'] if s['foil_word'] == 'ORIG']
foil_train_ADJ = [s for s in foil_train['annotations'] if s['POS'] == 'ADJ'] + [s for s in foil_train['annotations'] if s['foil_word'] == 'ORIG']
foil_train_ADV = [s for s in foil_train['annotations'] if s['POS'] == 'ADV'] + [s for s in foil_train['annotations'] if s['foil_word'] == 'ORIG']
foil_train_PP = [s for s in foil_train['annotations'] if s['POS'] == 'PP'] + [s for s in foil_train['annotations'] if s['foil_word'] == 'ORIG']




random.shuffle(foil_train_VERB)
random.shuffle(foil_train_ADJ)
random.shuffle(foil_train_ADV)
random.shuffle(foil_train_PP)

ourimagetrainfeats = [(int(w.strip().split()[0].split('.jpg')[0].split('COCO_train2014_')[1]), np.array(map(float, w.strip().split()[1:]))) 
                      for w in open('data/mscoco_boc_gt_train2014.txt')]
ourimagetrainfeats = dict(ourimagetrainfeats)

training_annotations_VERB = [l['caption'] for l in foil_train_VERB] 
training_annotations_ADV = [l['caption'] for l in foil_train_ADV]
training_annotations_PP = [l['caption'] for l in foil_train_PP]
training_annotations_ADJ = [l['caption'] for l in foil_train_ADJ]

tf_vectorizer = CountVectorizer(max_features=None, lowercase=True)
tf_model_VERB = tf_vectorizer.fit(training_annotations_VERB)
tf_model_ADV = tf_vectorizer.fit(training_annotations_ADV)
tf_model_PP = tf_vectorizer.fit(training_annotations_PP)
tf_model_ADJ = tf_vectorizer.fit(training_annotations_ADJ)

training_feats_VERB = tf_model_VERB.transform(training_annotations_VERB)
training_feats_ADV = tf_model_ADV.transform(training_annotations_ADV)
training_feats_PP = tf_model_PP.transform(training_annotations_PP)
training_feats_ADJ = tf_model_ADJ.transform(training_annotations_ADJ)

training_y_VERB = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_train_VERB]
training_y_ADJ = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_train_ADJ]
training_y_PP = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_train_PP]
training_y_ADV = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_train_ADV]


training_image_feats_VERB = [ourimagetrainfeats[i['image_id']] for i in foil_train_VERB]
training_image_feats_sparse_VERB = ss.csr_matrix(np.array(training_image_feats_VERB))
training_image_feats_ADJ = [ourimagetrainfeats[i['image_id']] for i in foil_train_ADJ]
training_image_feats_sparse_ADJ = ss.csr_matrix(np.array(training_image_feats_ADJ))
training_image_feats_ADV = [ourimagetrainfeats[i['image_id']] for i in foil_train_ADV]
training_image_feats_sparse_ADV = ss.csr_matrix(np.array(training_image_feats_ADV))
training_image_feats_PP = [ourimagetrainfeats[i['image_id']] for i in foil_train_PP]
training_image_feats_sparse_PP = ss.csr_matrix(np.array(training_image_feats_PP))



print len(training_image_feats_VERB), len(training_annotations_ADV), len(training_annotations_ADJ), \
         len(training_annotations_PP)



foil_test = json.load(open('foil_new/FOIL-LINGv0.5_test2017.json') )

foil_test_VERB = [s for s in foil_test['annotations'] if s['POS'] == 'VERB'] + [s for s in foil_test['annotations'] if s['foil_word'] == 'ORIG']
foil_test_PP = [s for s in foil_test['annotations'] if s['POS'] == 'PP'] + [s for s in foil_test['annotations'] if s['foil_word'] == 'ORIG']
foil_test_ADJ = [s for s in foil_test['annotations'] if s['POS'] == 'ADJ'] + [s for s in foil_test['annotations'] if s['foil_word'] == 'ORIG']
foil_test_ADV = [s for s in foil_test['annotations'] if s['POS'] == 'ADV'] + [s for s in foil_test['annotations'] if s['foil_word'] == 'ORIG']

random.shuffle(foil_test_VERB)
random.shuffle(foil_test_ADJ)
random.shuffle(foil_test_ADV)
random.shuffle(foil_test_PP)


testing_annotations_VERB = [l['caption'] for l in foil_test_VERB]
testing_annotations_ADV = [l['caption'] for l in foil_test_ADV]
testing_annotations_ADJ = [l['caption'] for l in foil_test_ADJ]
testing_annotations_PP = [l['caption'] for l in foil_test_PP]

ourimagetestfeats = [(int(w.strip().split()[0].split('.jpg')[0].split('COCO_val2014_')[1]),
                     map(float, w.strip().split()[1:])) 
                    for w in open('data/mscoco_boc_gt_val2014.txt')]
ourimagetestfeats = dict(ourimagetestfeats)



testing_feats_VERB = tf_model_VERB.transform(testing_annotations_VERB)
testing_feats_PP = tf_model_PP.transform(testing_annotations_PP)
testing_feats_ADJ = tf_model_ADJ.transform(testing_annotations_ADJ)
testing_feats_ADV = tf_model_ADV.transform(testing_annotations_ADV)



testing_y_VERB = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_test_VERB]
testing_y_ADJ = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_test_ADJ]
testing_y_ADV = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_test_ADV]
testing_y_PP = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_test_PP]

testing_image_feats_VERB = [ourimagetestfeats[i['image_id']] for i in foil_test_VERB]
testing_image_feats_sparse_VERB = ss.csr_matrix(np.array(testing_image_feats_VERB))
testing_image_feats_ADJ = [ourimagetestfeats[i['image_id']] for i in foil_test_ADJ]
testing_image_feats_sparse_ADJ = ss.csr_matrix(np.array(testing_image_feats_ADJ))
testing_image_feats_ADV = [ourimagetestfeats[i['image_id']] for i in foil_test_ADV]
testing_image_feats_sparse_ADV = ss.csr_matrix(np.array(testing_image_feats_ADV))
testing_image_feats_PP = [ourimagetestfeats[i['image_id']] for i in foil_test_PP]
testing_image_feats_sparse_PP = ss.csr_matrix(np.array(testing_image_feats_PP))

print len(training_y_VERB),  sum(training_y_VERB)

60262 53381 73057 77002
60262 7925


In [4]:
from tqdm import tqdm

def dump(dirtrain, dirtest, dtypetr, dtypete):
    for r in tqdm(range(len(dtypetr))):
        np.save(dirtrain + '/' + str(r+1)+'.npy', dtypetr[r])
    print 'done with training'
    for r in tqdm(range(len(dtypete))):
        np.save(dirtest + '/' + str(r+1)+'.npy', dtypete[r])
    print 'done with testing'
    
    


In [22]:
%mkdir train_img_verb test_img_verb
dump('train_img_verb', 'test_img_verb',  training_image_feats_VERB, testing_image_feats_VERB)


%mkdir train_img_adj test_img_adj
dump('train_img_adj', 'test_img_adj',  training_image_feats_ADJ, testing_image_feats_ADJ)

%mkdir train_img_adv test_img_adv
dump('train_img_adv', 'test_img_adv',  training_image_feats_ADV, testing_image_feats_ADV)

%mkdir train_img_pp test_img_pp
dump('train_img_pp', 'test_img_pp',  training_image_feats_PP, testing_image_feats_PP)



  3%|▎         | 1151/33616 [00:00<00:02, 11504.47it/s]

done with training


100%|██████████| 33616/33616 [00:02<00:00, 11416.34it/s]

done with testing



  3%|▎         | 1154/42163 [00:00<00:03, 11535.97it/s]

...

done with testing



  2%|▏         | 1149/46018 [00:00<00:03, 11487.24it/s]

done with training


100%|██████████| 46018/46018 [00:04<00:00, 10473.37it/s]

done with testing


In [37]:

training_y_VERB_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_train_VERB]
training_y_ADJ_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_train_ADJ]
training_y_PP_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_train_PP]
training_y_ADV_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_train_ADV]

def write_stuff(annots, ys, filename):
    with open(filename, 'w') as f:
        for x,y in zip(annots, ys):
            f.write('%s\t%s\n' % (y, x.replace('\n', ' ').strip('\n').strip()))
            
    print 'done'
    
    

testing_y_VERB_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_test_VERB]
testing_y_ADJ_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_test_ADJ]
testing_y_PP_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_test_ADV]
testing_y_ADV_mod = ['REAL' if f['foil_word'] == 'ORIG' else 'FAKE' for f in foil_test_PP]



In [36]:
testing_annotations_PP[13686].replace('\n', ' ')

u'A cat steeping into a gross litter box on a table!    Yikes'

In [38]:
write_stuff(training_annotations_VERB, training_y_VERB, 'VerbOnlyTraining.txt')
write_stuff(training_annotations_ADJ, training_y_ADJ, 'AdjOnlyTraining.txt')
write_stuff(training_annotations_ADV, training_y_ADV, 'AdvOnlyTraining.txt')
write_stuff(training_annotations_PP, training_y_PP, 'PPOnlyTraining.txt')


write_stuff(testing_annotations_VERB, testing_y_VERB, 'VerbOnlyTesting.txt')
write_stuff(testing_annotations_ADJ, testing_y_ADJ, 'AdjOnlyTesting.txt')
write_stuff(testing_annotations_ADV, testing_y_ADV, 'AdvOnlyTesting.txt')
write_stuff(testing_annotations_PP, testing_y_PP, 'PPOnlyTesting.txt')



done
done
done
done
done
done
done
done


In [24]:
sio.mmwrite('data_new/train_feats_VERB.mtx', training_feats_VERB)
sio.mmwrite('data_new/test_feats_VERB.mtx', testing_feats_VERB)
sio.mmwrite('data_new/train_feats_ADJ.mtx', training_feats_ADJ)
sio.mmwrite('data_new/test_feats_ADJ.mtx', testing_feats_ADJ)
sio.mmwrite('data_new/train_feats_ADV.mtx', training_feats_ADV)
sio.mmwrite('data_new/test_feats_ADV.mtx', testing_feats_ADV)
sio.mmwrite('data_new/train_feats_PP.mtx', training_feats_PP)
sio.mmwrite('data_new/test_feats_PP.mtx', testing_feats_PP)


sio.mmwrite('data_new/train_image_feats_VERB.mtx', training_image_feats_sparse_VERB)
sio.mmwrite('data_new/test_image_feats_VERB.mtx', testing_image_feats_sparse_VERB)
sio.mmwrite('data_new/train_image_feats_PP.mtx', training_image_feats_sparse_PP)
sio.mmwrite('data_new/test_image_feats_PP.mtx', testing_image_feats_sparse_PP)
sio.mmwrite('data_new/train_image_feats_ADJ.mtx', training_image_feats_sparse_ADJ)
sio.mmwrite('data_new/test_image_feats_ADJ.mtx', testing_image_feats_sparse_ADJ)
sio.mmwrite('data_new/train_image_feats_ADV.mtx', training_image_feats_sparse_ADV)
sio.mmwrite('data_new/test_image_feats_ADV.mtx', testing_image_feats_sparse_ADV)




sio.mmwrite('data_new/train_y_VERB.mtx', ss.csr_matrix(np.array(training_y_VERB)))
sio.mmwrite('data_new/test_y_VERB.mtx', ss.csr_matrix(np.array(testing_y_VERB)))
sio.mmwrite('data_new/train_y_ADJ.mtx', ss.csr_matrix(np.array(training_y_ADJ)))
sio.mmwrite('data_new/test_y_ADJ.mtx', ss.csr_matrix(np.array(testing_y_ADJ)))
sio.mmwrite('data_new/train_y_ADV.mtx', ss.csr_matrix(np.array(training_y_ADV)))
sio.mmwrite('data_new/test_y_ADV.mtx', ss.csr_matrix(np.array(testing_y_ADV)))
sio.mmwrite('data_new/train_y_PP.mtx', ss.csr_matrix(np.array(training_y_PP)))
sio.mmwrite('data_new/test_y_PP.mtx', ss.csr_matrix(np.array(testing_y_PP)))





{u'POS': u'NA',
 u'Source': u'NA',
 u'caption': u'A young man riding a skateboard up into the air.',
 u'foil_id': 4118297,
 u'foil_word': u'ORIG',
 u'id': 732811,
 u'image_id': 491151,
 u'target_word': u'ORIG'}

In [26]:
foiled_stuff = [a for a in foil_train['annotations'] if a['foil_word'] != 'ORIG' ]

In [27]:
m = [a['POS'] for a in foiled_stuff]
set(m) 



{u'ADJ', u'ADV', u'PP', u'VERB'}

In [28]:
print testing_feats_VERB[0]

  (0, 1244)	1
  (0, 3149)	1
  (0, 3648)	1
  (0, 5856)	1
  (0, 7662)	1
  (0, 8138)	1
  (0, 8729)	1


9834

In [34]:
tf_model_VERB.get_feature_names()[-1]

u'zucchinni'

In [4]:
len([s for s in foil_train['annotations'] if s['POS'] == 'VERB'])

7925

In [5]:
len([s for s in foil_train['annotations'] if s['POS'] == 'ADJ'])

20720

In [6]:
len([s for s in foil_train['annotations'] if s['POS'] == 'ADV'])

1044

In [7]:
len([s for s in foil_train['annotations'] if s['POS'] == 'PP'])

24665

In [9]:
len(foil_train_VERB), len(foil_train_ADJ), len(foil_train_ADV), len(foil_train_PP)


(60262, 73057, 53381, 77002)

In [10]:
len(foil_test_VERB), len(foil_test_ADJ), len(foil_test_ADV), len(foil_test_PP)

(33616, 42163, 30738, 46018)